In [1]:
import pandas as pd
import numpy as np

data = pd.read_csv("../../data/master/rumdetect2017/rumor_detection_acl2017/twitter15/source_tweets.txt", header=None, delimiter="\t", lineterminator="\n")
data.columns = ["tweet_id", "tweet_text"]
print(data.head())

             tweet_id                                         tweet_text
0  731166399389962242  🔥ca kkk grand wizard 🔥 endorses @hillaryclinto...
1  714598641827246081  an open letter to trump voters from his top st...
2  691809004356501505  america is a nation of second chances —@potus ...
3  693204708933160960  brandon marshall visits and offers advice, sup...
4  551099691702956032  rip elly may clampett: so sad to learn #beverl...


In [2]:
labels = pd.read_csv("../../data/master/rumdetect2017/rumor_detection_acl2017/twitter15/label.txt", header=None, delimiter=":", lineterminator="\n")
labels.columns = ["label", "tweet_id"]
print(labels.head())

        label            tweet_id
0  unverified  731166399389962242
1  unverified  714598641827246081
2   non-rumor  691809004356501505
3   non-rumor  693204708933160960
4        true  551099691702956032


In [3]:
label_dict = {}
labs = {}
for i, l in labels.iterrows():
    label_dict[l["tweet_id"]] = l["label"]
    if l["label"] not in labs:
        labs[l["label"]] = 1
    else:
        labs[l["label"]] += 1
print(labs)
len(label_dict)

{'unverified': 374, 'non-rumor': 374, 'true': 372, 'false': 370}


1490

In [4]:
length = data.shape[0]
length

1490

In [5]:
length = labels.shape[0]
length

1490

In [7]:
data['label'] = data['tweet_id'].map(lambda a: label_dict[a])
data.head()

,tweet_id,tweet_text,label
0,731166399389962242,🔥ca kkk grand wizard 🔥 endorses @hillaryclinto...,unverified
1,714598641827246081,an open letter to trump voters from his top st...,unverified
2,691809004356501505,america is a nation of second chances —@potus ...,non-rumor
3,693204708933160960,"brandon marshall visits and offers advice, sup...",non-rumor
4,551099691702956032,rip elly may clampett: so sad to learn #beverl...,true


In [8]:
import random

random.seed(77)

weight = [0, 0, 0, 0, 0, 0, 1, 1, 2, 2]
tvt = []
for i in range(length):
    gacha = random.sample(weight, 1)[0]
    if gacha == 0:
        tvt.append("training")
    elif gacha == 1:
        tvt.append("validation")
    else:
        tvt.append("test")

print(f"Training : {tvt.count('training')} - {round(tvt.count('training')/length, 2)}")
print(f"Validation : {tvt.count('validation')} - {round(tvt.count('validation')/length, 2)}")
print(f"Test : {tvt.count('test')} - {round(tvt.count('test')/length, 2)}")

Training : 894 - 0.6
Validation : 302 - 0.2
Test : 294 - 0.2


In [9]:
data['tvt'] = pd.Series(tvt)
data.head()

,tweet_id,tweet_text,label,tvt
0,731166399389962242,🔥ca kkk grand wizard 🔥 endorses @hillaryclinto...,unverified,training
1,714598641827246081,an open letter to trump voters from his top st...,unverified,training
2,691809004356501505,america is a nation of second chances —@potus ...,non-rumor,training
3,693204708933160960,"brandon marshall visits and offers advice, sup...",non-rumor,training
4,551099691702956032,rip elly may clampett: so sad to learn #beverl...,true,training


In [10]:
data[100:105]

,tweet_id,tweet_text,label,tvt
100,500327120770301952,how many white celebrities have been caught sh...,unverified,training
101,693821069879160832,from @otlonespn: baylor faces accusations of i...,non-rumor,training
102,532281981795848192,manager bans teen slang from work including 'c...,false,validation
103,767528171520524288,japanese prime minister shinzo abe at #rio2016...,non-rumor,training
104,496698590153416704,justin bieber ringtone saves russian fisherman...,false,validation


In [12]:
label_count = data['label'].value_counts()
label_count

unverified    374
non-rumor     374
true          372
false         370
Name: label, dtype: int64

In [13]:
combination = data.apply(lambda row: f"{row['label']}_{row['tvt']}", axis=1).value_counts()
combination

unverified_training      230
non-rumor_training       229
false_training           218
true_training            217
true_validation           86
non-rumor_test            80
false_test                77
unverified_validation     76
false_validation          75
true_test                 69
unverified_test           68
non-rumor_validation      65
dtype: int64

In [14]:
data.to_csv("../../data/processed/twitter15_dataset_with_tvt.csv", index=False)